In [91]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
#data from 1973 - 2019
data = gpd.read_file('./ds2783.gdb/' , delim_whitespace = True)
data.info() #see what our data looks like
data.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 443421 entries, 0 to 443420
Data columns (total 40 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   RPT_YR          443421 non-null  int64   
 1   SURVEY_ID1      376258 non-null  object  
 2   SURVEY_ID2      375963 non-null  object  
 3   SURVEY_ID3      370896 non-null  object  
 4   DMG_TYPE1       443322 non-null  object  
 5   DMG_TYPE2       375142 non-null  object  
 6   DMG_TYPE3       374119 non-null  object  
 7   SEVERITY1       401858 non-null  object  
 8   SEVERITY2       375072 non-null  object  
 9   SEVERITY3       374012 non-null  object  
 10  PATTERN1        61754 non-null   object  
 11  PATTERN2        300 non-null     object  
 12  PATTERN3        31 non-null      object  
 13  TPA1            419113 non-null  float64 
 14  TPA2            376731 non-null  float64 
 15  TPA3            375771 non-null  float64 
 16  TPA_Total       440324 non-nul

,RPT_YR,SURVEY_ID1,SURVEY_ID2,SURVEY_ID3,DMG_TYPE1,DMG_TYPE2,DMG_TYPE3,SEVERITY1,SEVERITY2,SEVERITY3,...,ACRES,NOTES,no_years,Date,ADS_ID,Shape_Length0,Shape_Area0,Shape_Length,Shape_Area,geometry
0,2001,,,,Mortality,No Data,No Data,No data,No data,No data,...,103.120829,"est 100 ac, pockets & indiv trees homes, from ...",1.0,None,None,2852.471058,4.173169e+05,2852.471058,4.173169e+05,"MULTIPOLYGON (((218050.693 -404106.915, 218260..."
1,1994,,,,Mortality,Mortality,No Data,No data,No data,No data,...,10271.778222,"pine = 50%, fir = 50%, from BDF survey",1.0,None,None,30684.592396,4.156858e+07,30684.592396,4.156858e+07,"MULTIPOLYGON (((218401.223 -407776.155, 218422..."
2,2004,,,,Mortality,No Data,No Data,No data,No data,No data,...,0.718024,p!3!,1.0,None,None,191.341969,2.905751e+03,191.341969,2.905751e+03,"MULTIPOLYGON (((186752.913 -403946.705, 186747..."
3,2004,,,,Mortality,No Data,No Data,No data,No data,No data,...,0.718054,mc!10!,1.0,None,None,191.345709,2.905874e+03,191.345709,2.905874e+03,"MULTIPOLYGON (((207053.633 -403337.415, 207048..."
4,2004,,,,Mortality,No Data,No Data,No data,No data,No data,...,0.717917,2 SP,1.0,None,None,191.344635,2.905317e+03,191.344635,2.905317e+03,"MULTIPOLYGON (((164632.113 -404676.935, 164626..."


Sort Chronologically

In [92]:
data.set_index('RPT_YR' , inplace = True)
data.sort_values(by = 'RPT_YR' , ascending = True , inplace = True) #sort by year first, then sort by Date
data.reset_index('RPT_YR' , inplace = True)

data['datetime'] = pd.to_datetime(data['Date'] , errors = 'coerce') #create new datetime column
data.set_index('datetime' , inplace = True) #set new column to index
data.sort_values(by='datetime', ascending = True, inplace = True) #sort values
data.reset_index('datetime' , inplace = True) #reset index to numbered list
data.drop('datetime' , axis = 1 , inplace = True) #delete the datetime column

print(len(data) , data.index)
data.tail

443421 RangeIndex(start=0, stop=443421, step=1)


<bound method NDFrame.tail of         RPT_YR SURVEY_ID1 SURVEY_ID2 SURVEY_ID3      DMG_TYPE1 DMG_TYPE2  \
0         1973        DFB         -1         -1      Mortality   No Data   
1         1973        DFB         -1         -1      Mortality   No Data   
2         1973        DFB         -1         -1      Mortality   No Data   
3         1973        DFB         -1         -1      Mortality   No Data   
4         1973        DFB         -1         -1      Mortality   No Data   
...        ...        ...        ...        ...            ...       ...   
443416    2019       None       None       None  Mortality - 2      None   
443417    2019       None       None       None  Mortality - 2      None   
443418    2019       None       None       None  Mortality - 2      None   
443419    2019       None       None       None  Mortality - 2      None   
443420    2019       None       None       None  Mortality - 2      None   

       DMG_TYPE3          SEVERITY1 SEVERITY2 SEVERITY3  

Convert Geometry data to coordinates

In [93]:
data = data.set_crs(epsg = 3310)
data = data.to_crs(epsg = 4326)
data.geometry.bounds

,minx,miny,maxx,maxy
0,-120.896076,40.058487,-120.892896,40.061901
1,-120.931935,39.822156,-120.925507,39.827273
2,-121.009019,39.810196,-121.005204,39.815451
3,-121.033416,39.796240,-121.030554,39.799726
4,-121.039081,39.780196,-121.033818,39.784047
...,...,...,...,...
443416,-123.400168,41.956071,-123.371965,41.983281
443417,-120.188302,41.854432,-120.184275,41.856548
443418,-118.983255,37.703216,-118.981650,37.703898
443419,-120.279755,41.284719,-120.261646,41.301275


Only keep columns we are interested in (mostly location and date)

In [94]:
columns_to_drop = ['SURVEY_ID1', 'SURVEY_ID2', 'SURVEY_ID3', 'DMG_TYPE1', 'DMG_TYPE2', 'DMG_TYPE3', 
'SEVERITY2', 'SEVERITY3', 'PATTERN1', 'PATTERN2', 'PATTERN3', 'TPA1', 'TPA2', 'TPA3', 'NO_TREES1', 
'NO_TREES2', 'NO_TREES3', 'DCA1', 'DCA2', 'DCA3', 'HOST1', 'HOST2', 'HOST3', 'FOR_TYPE1', 'FOR_TYPE2',
 'FOR_TYPE3', 'NOTES', 'no_years', 'ADS_ID', 'Shape_Length0', 'Shape_Area0', 'Shape_Length', 'Shape_Area' , 'NO_TREES_TOTAL'] #list of columns to delete

data.drop(columns_to_drop , axis = 1 , inplace = True) #delete columns
data.info() #display info

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 443421 entries, 0 to 443420
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype   
---  ------     --------------   -----   
 0   RPT_YR     443421 non-null  int64   
 1   SEVERITY1  401858 non-null  object  
 2   TPA_Total  440324 non-null  float64 
 3   ACRES      443421 non-null  float64 
 4   Date       81386 non-null   object  
 5   geometry   443421 non-null  geometry
dtypes: float64(2), geometry(1), int64(1), object(2)
memory usage: 20.3+ MB


Remove fires less than 100 acres, no severity information, and before 1984

In [95]:
print('entries:' , len(data)) #display number of rows
data = data.loc[data['ACRES'] > 100] #assign dataframe with the same frame but w/ acres > 10
print('entries:' , len(data))
missing_severity = data.loc[data['SEVERITY1'].isnull()] #find entries where severity is null
missing_severity1 = data.loc[data['SEVERITY1'] == 'No data'] #find entries where severity is null
data.drop(missing_severity.index , inplace = True) #drop said entries
data.drop(missing_severity1.index , inplace = True)
print('entries:' , len(data))
data = data.loc[data["RPT_YR"] > 1984] #loc function allows for sorting without reindexing
print('entries:' , len(data))
data.head()

entries: 443421
entries: 53314
entries: 16018
entries: 16018


,RPT_YR,SEVERITY1,TPA_Total,ACRES,Date,geometry
6764,1997,More then 50 % defoliation,0.0,3708.245532,None,"MULTIPOLYGON (((-119.29728 37.87035, -119.2975..."
6765,1997,More then 50 % defoliation,0.0,109.554970,None,"MULTIPOLYGON (((-119.62179 37.69426, -119.6220..."
6766,1997,More then 50 % defoliation,0.0,2924.703512,None,"MULTIPOLYGON (((-119.49148 37.81720, -119.4917..."
6767,1997,More then 50 % defoliation,0.0,150.956996,None,"MULTIPOLYGON (((-119.63627 37.53037, -119.6359..."
6768,1997,Equal to or Less than 50 % defoliation,0.0,47231.703213,None,"MULTIPOLYGON (((-119.87979 37.99364, -119.8795..."


Export cleaned data to a .geojson

In [96]:
data.info
data.to_file('Clean_tree.geojson', driver = 'GeoJSON' , encoding = 'utf-8')

In [97]:
data.head


<bound method NDFrame.head of         RPT_YR                               SEVERITY1  TPA_Total  \
6764      1997              More then 50 % defoliation   0.000000   
6765      1997              More then 50 % defoliation   0.000000   
6766      1997              More then 50 % defoliation   0.000000   
6767      1997              More then 50 % defoliation   0.000000   
6768      1997  Equal to or Less than 50 % defoliation   0.000000   
...        ...                                     ...        ...   
443413    2019                       Moderate (11-29%)  11.371073   
443414    2019                       Very Light (1-3%)   0.872343   
443415    2019                       Moderate (11-29%)  11.371073   
443416    2019                       Moderate (11-29%)  11.371073   
443419    2019                       Moderate (11-29%)  11.371073   

               ACRES  Date                                           geometry  
6764     3708.245532  None  MULTIPOLYGON (((-119.29728 37.870